In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset

resize dataset pixel size

In [ ]:
from PIL import Image
import os

# 입력 경로에서 모든 이미지 파일을 가져오기
for i in range(3,0,-1):

#    1~13, 25

  print(i)

  image_file = "/content/gdrive/MyDrive/aircon/test/2/muni"+str(i)+".jpg"
  image = Image.open(image_file)

  # 이미지 크기 조절
  resized_image = image.resize((224,224))

  # 새로운 경로에 이미지 저장
  resized_image.save(image_file)

#tower는 1


3
2
1


그냥 이미지 원본들을 데이터셋으로 만들기

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=True):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)

        self.data = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_path):
                self.data.append(os.path.join(class_path, file_name))
                self.labels.append(i)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return {'data': img, 'label': label}


In [ ]:
#split dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 모든 이미지를 동일한 크기로 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_dataset = CustomDataset("/content/gdrive/MyDrive/AIR/aug", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
test_dataset = CustomDataset("/content/gdrive/MyDrive/AIR/test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
print(len(train_loader.dataset))

101


이미지 aug

In [ ]:
pip install opencv-python Augmentor

In [ ]:
import os
import cv2
from tqdm import tqdm
from Augmentor import Pipeline

def apply_augmentation(input_path, output_path):
    # Input path에 있는 모든 이미지 파일 가져오기
    image_files = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Augmentor 파이프라인 생성
    pipeline = Pipeline(input_path, output_directory=output_path)

    # 필요한 augmentation 작업 추가 (회전, 뒤집기, 밝기 조절, 색상 변환 등)
    pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    pipeline.flip_left_right(probability=0.5)
    pipeline.random_brightness(probability=0.7, min_factor=0.7, max_factor=1.3)
    pipeline.random_color(probability=0.5, min_factor=0.7, max_factor=1.3)

    # Augmentation 적용 및 저장
    pipeline.sample(len(image_files), multi_threaded=True)

if __name__ == "__main__":
    # 경로 1 및 경로 2 지정
    input_path = "/content/gdrive/MyDrive/AIR/train/2"
    output_path = "/content/gdrive/MyDrive/AIR/aug/2"

    # Augmentation 적용 및 저장
    apply_augmentation(input_path, output_path)


Initialised with 30 image(s) found.
Output directory set to /content/gdrive/MyDrive/AIR/aug/2.

Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x79DF42D0D480>: 100%|██████████| 30/30 [00:00<00:00, 124.57 Samples/s]


학습 준비 및 학습

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cuda:0 is available


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(128, 3)  # Assuming there are 10 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        # Reshape before fully connected layers
        x = x.view(-1, 32 * 56 * 56)

        # Fully connected layers
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
class CL3CNN(nn.Module):
    def __init__(self):
        super(CL3CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.relu4 = nn.ReLU()  # Corrected the name here

        self.fc2 = nn.Linear(256, 3)  # Assuming there are 10 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))

        # Reshape before fully connected layers
        x = x.view(-1, 64 * 28 * 28)

        # Fully connected layers
        x = self.relu4(self.fc1(x))  # Corrected the name here
        x = self.fc2(x)

        return x

In [ ]:
class CL3FC3CNN(nn.Module):
    def __init__(self):
        super(CL3FC3CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.relu4 = nn.ReLU()  # Corrected the name here
        self.fc2=nn.Linear(256,64)
        self.relu5=nn.ReLU()
        self.fc3 = nn.Linear(64, 3)  # Assuming there are 10 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))

        # Reshape before fully connected layers
        x = x.view(-1, 64 * 28 * 28)

        # Fully connected layers
        x = self.relu4(self.fc1(x))  # Corrected the name here
        x = self.fc2(x)
        x = self.relu5(x)
        x = self.fc3(x)

        return x

In [ ]:
class CL4CNN(nn.Module):
    def __init__(self):
        super(CL4CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 14 * 14, 256)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)  # Assuming there are 3 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.pool4(self.relu4(self.conv4(x)))

        # Reshape before fully connected layers
        x = x.view(-1, 128 * 14 * 14)

        # Fully connected layers
        x = self.relu5(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
class CL5CNN(nn.Module):
    def __init__(self):
        super(CL5CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 7 * 7, 256)  # Corrected the name here
        self.relu6 = nn.ReLU()
        self.fc2 = nn.Linear(256, 3)  # Assuming there are 3 classes

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.pool4(self.relu4(self.conv4(x)))
        x = self.pool5(self.relu5(self.conv5(x)))  # Corrected the name here

        # Reshape before fully connected layers
        x = x.view(-1, 256 * 7 * 7)

        # Fully connected layers
        x = self.relu6(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
class CL4FC3CNN(nn.Module):
    def __init__(self):
        super(CL4FC3CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 14 * 14, 256)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(256, 64)
        self.relu6=nn.ReLU()
        self.fc3=nn.Linear(64,3)


    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.pool4(self.relu4(self.conv4(x)))

        # Reshape before fully connected layers
        x = x.view(-1, 128 * 14 * 14)

        # Fully connected layers
        x = self.relu5(self.fc1(x))
        x=self.relu6(self.fc2(x))
        x=self.fc3(x)

        return x


In [ ]:
# 모델 인스턴스 생성
model = CL4FC3CNN()
#CL3CNN
#CL3FC3CNN
#CL4CNN
#CL5CNN

In [ ]:
# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs=10

In [ ]:
for epoch in range(epochs):
    model.train()  # 학습 모드로 설정


    for batch in train_loader:
        data, labels = batch['data'], batch['label']

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 각 에폭 끝에 손실 출력 또는 로깅 추가
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


    #  모델 성능 평가
    model.eval()  # 평가 모드로 설정

    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_loader:
            data, labels = batch['data'], batch['label']

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'{epoch+1} epoch - Test Accuracy: {accuracy * 100:.2f}%')



Epoch [1/10], Loss: 0.693561851978302
1 epoch - Test Accuracy: 40.00%
Epoch [2/10], Loss: 0.5382905602455139
2 epoch - Test Accuracy: 20.00%
Epoch [3/10], Loss: 1.098335862159729
3 epoch - Test Accuracy: 30.00%
Epoch [4/10], Loss: 0.04717760160565376
4 epoch - Test Accuracy: 20.00%
Epoch [5/10], Loss: 0.014002463780343533
5 epoch - Test Accuracy: 20.00%
Epoch [6/10], Loss: 1.807303786277771
6 epoch - Test Accuracy: 30.00%
Epoch [7/10], Loss: 0.003925358410924673
7 epoch - Test Accuracy: 20.00%
Epoch [8/10], Loss: 0.9944594502449036
8 epoch - Test Accuracy: 0.00%
Epoch [9/10], Loss: 0.40150535106658936
9 epoch - Test Accuracy: 40.00%
Epoch [10/10], Loss: 0.010020076297223568
10 epoch - Test Accuracy: 40.00%
